In [1]:
# Bibliotecas

import pandas as pd
import numpy as np
import joblib # Para salvar o modelo em disco
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder , LabelEncoder
from sklearn.compose import ColumnTransformer
import warnings
import watermark
warnings.filterwarnings('ignore')

LabelEncoder: Rotula a classe com um número (Geralmente sequencial) Ex: 0 para azul, 1 para amarelo e 2 para Vermelho.

    Vantagens: Pode ser útil quando as classes possuem uma ordem hierarquica (ex: Junior, pleno e Sênior)
    Desvantagens:Caso as classes não possuam ordem hierarquica o modelo pode dar mais peso para a classe que tem um valor maior, isso pode enviesar o modelo.


    
OneHotEncoder : Cria uma coluna para cada categoria e faz a classificação binária (0 e 1) ex:Azul, amarelo e vermelho : Cria-se 3 colunas, uma para cada classe e classifica 0 para falso e 1 para verdadeiro

    Vantagens: Não enviesa o modelo.
    Desvantagens: Não recomendado quando há muitas classes, visto que pode aumentar muito a dimensionalidade e consumo computacional.




In [2]:
# Autor do projeto e versão dos pacotes
%reload_ext watermark
%watermark -a "Guilherme"
%watermark --iversions

Author: Guilherme

numpy    : 1.26.4
watermark: 2.5.0
joblib   : 1.4.2
sklearn  : 1.5.1
pandas   : 2.2.2



# Importando e explorando os dados

In [3]:
df= pd.read_csv('dataset.csv')

In [4]:
df.head(20)

,Modelo,Kilometragem,Preco_Venda,Idade_Veiculo
0,Volkswagen Polo,79000,18000,6
1,Volkswagen Polo,65000,34000,3
2,Volkswagen Polo,57000,26100,5
3,Volkswagen Polo,72500,40000,2
4,Volkswagen Polo,44000,31500,4
5,Chevrolet Onix,69000,29400,5
6,Chevrolet Onix,62000,32000,5
7,Chevrolet Onix,61000,19300,6
8,Chevrolet Onix,71000,12000,8
9,Hyundai HB20,64000,22000,6


In [5]:
# Estrutura da base
df.shape

(13, 4)

In [6]:
# Metadados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Modelo         13 non-null     object
 1   Kilometragem   13 non-null     int64 
 2   Preco_Venda    13 non-null     int64 
 3   Idade_Veiculo  13 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 548.0+ bytes


In [7]:
# Analises descritivas
round(df.describe(),2)

,Kilometragem,Preco_Venda,Idade_Veiculo
count,13.00,13.00,13.00
mean,65807.69,26023.08,5.31
std,10044.77,8003.66,1.65
min,44000.00,12000.00,2.00
25%,61000.00,20000.00,5.00
50%,65000.00,26100.00,5.00
75%,72500.00,32000.00,6.00
max,81000.00,40000.00,8.00


# Aplicando o OneHotEncoder

In [8]:
df_dummies = pd.get_dummies(df['Modelo'])

In [9]:
df_dummies

,Chevrolet Onix,Hyundai HB20,Volkswagen Polo
0,False,False,True
1,False,False,True
2,False,False,True
3,False,False,True
4,False,False,True
5,True,False,False
6,True,False,False
7,True,False,False
8,True,False,False
9,False,True,False


In [10]:
# Concatenando os dataframes

df_1 = pd.concat([df, df_dummies],axis=1)

In [11]:
df_1

,Modelo,Kilometragem,Preco_Venda,Idade_Veiculo,Chevrolet Onix,Hyundai HB20,Volkswagen Polo
0,Volkswagen Polo,79000,18000,6,False,False,True
1,Volkswagen Polo,65000,34000,3,False,False,True
2,Volkswagen Polo,57000,26100,5,False,False,True
3,Volkswagen Polo,72500,40000,2,False,False,True
4,Volkswagen Polo,44000,31500,4,False,False,True
5,Chevrolet Onix,69000,29400,5,True,False,False
6,Chevrolet Onix,62000,32000,5,True,False,False
7,Chevrolet Onix,61000,19300,6,True,False,False
8,Chevrolet Onix,71000,12000,8,True,False,False
9,Hyundai HB20,64000,22000,6,False,True,False


In [12]:
# Excluindo a variável modelo, visto que já temos o mapeamendo
df_1.drop(columns='Modelo', inplace=True)

In [13]:
df_1.head()

,Kilometragem,Preco_Venda,Idade_Veiculo,Chevrolet Onix,Hyundai HB20,Volkswagen Polo
0,79000,18000,6,False,False,True
1,65000,34000,3,False,False,True
2,57000,26100,5,False,False,True
3,72500,40000,2,False,False,True
4,44000,31500,4,False,False,True


### Criando o modelo de machine learning

In [14]:
x = df_1.drop('Preco_Venda', axis=1)
y =df_1['Preco_Venda']

In [15]:
# Instanciando o modelo de regressão Linear
modelo1 = linear_model.LinearRegression()

In [16]:
# Treinando o modelo
modelo1.fit(x,y)

LinearRegression()

In [17]:
# Aplicando o modelo
modelo1.predict(x)

array([18105.0215775 , 35327.25262423, 24783.14553092, 40410.02746228,
       30974.55280508, 28487.61326992, 28866.17068713, 23431.87824739,
       11914.33779555, 25628.97126508, 19221.24546681, 19653.88251505,
       31495.90075306])

In [18]:
# Testando o modelo
modelo1.predict([[200000,0,0,0,1]])

array([44491.61864755])

In [19]:
# Calculando a métrica de Score
modelo1.score(x,y)

0.9355612290620992

# Explicando a métrica do Score (R²)

Valor entre 0 e 1:

0 significa que o modelo não explica nenhuma variabilidade dos dados.

1 significa que o modelo explica toda a variabilidade — ou seja, um ajuste perfeito (o que raramente acontece na prática).

Análise do resultado: 
93,5% da variação da target(Preco_Venda) pode ser explicada pelas demais variáveis

# Aplicando o LabelEncoder


In [20]:
# Criando uma coópia da base original
df_2 = df

In [21]:
# Instanciando o LabelEncoder
le = LabelEncoder()

In [22]:
# Aplicando o Label encoder na variável modelo
df_2['Modelo'] = le.fit_transform(df_2['Modelo'])

In [23]:
# Resultado
df_2

,Modelo,Kilometragem,Preco_Venda,Idade_Veiculo
0,2,79000,18000,6
1,2,65000,34000,3
2,2,57000,26100,5
3,2,72500,40000,2
4,2,44000,31500,4
5,0,69000,29400,5
6,0,62000,32000,5
7,0,61000,19300,6
8,0,71000,12000,8
9,1,64000,22000,6


In [24]:
# Separado as features(x) da Target(y)
x = df_2.drop(columns='Preco_Venda').values
y = df_2['Preco_Venda']

In [25]:
# Instanciando o modelo
modelo2 = linear_model.LinearRegression()

In [26]:
# Treinando o modelo de regressão logistíca
modelo2.fit(x,y)

LinearRegression()

In [27]:
# Aplicando o modelo
modelo2.predict(x)

array([20594.96255322, 35709.41910683, 26372.80065489, 40228.36867645,
       31766.57106881, 29143.34538167, 29442.06469435, 24645.7329058 ,
       14540.97907838, 22876.39285467, 17311.92569064, 17653.31919085,
       28014.11814344])

In [28]:
#Calculando a métrcica de Score(R²)
modelo2.score(x,y)

0.8757010116903795

Análise do resultado: 
87,5% da variação da target(Preco_Venda) pode ser explicada pelas demais variáveis

# Aplicando one hot encoder com transformer

In [30]:
# Criando uma cópia da base original
df_OHEncoder = df

In [31]:
# instanciando o enconding com transformer para deixar na mesma escala
OHEncoder = ColumnTransformer([('Modelo',OneHotEncoder(), [0])], remainder='passthrough')

In [32]:
OHEncoder

ColumnTransformer(remainder='passthrough',
                  transformers=[('Modelo', OneHotEncoder(), [0])])

In [33]:
# Resultado
df_OHEncoder

,Modelo,Kilometragem,Preco_Venda,Idade_Veiculo
0,2,79000,18000,6
1,2,65000,34000,3
2,2,57000,26100,5
3,2,72500,40000,2
4,2,44000,31500,4
5,0,69000,29400,5
6,0,62000,32000,5
7,0,61000,19300,6
8,0,71000,12000,8
9,1,64000,22000,6


In [35]:
# Defindo as features
x = df_OHEncoder.drop('Preco_Venda', axis=1)

In [36]:
# Defindo a target
y = df_OHEncoder['Preco_Venda']

In [37]:
# Aplicando a transformação
X = OHEncoder.fit_transform(x)

In [38]:
# Resultado
X

array([[0.00e+00, 0.00e+00, 1.00e+00, 7.90e+04, 6.00e+00],
       [0.00e+00, 0.00e+00, 1.00e+00, 6.50e+04, 3.00e+00],
       [0.00e+00, 0.00e+00, 1.00e+00, 5.70e+04, 5.00e+00],
       [0.00e+00, 0.00e+00, 1.00e+00, 7.25e+04, 2.00e+00],
       [0.00e+00, 0.00e+00, 1.00e+00, 4.40e+04, 4.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, 6.90e+04, 5.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, 6.20e+04, 5.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, 6.10e+04, 6.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, 7.10e+04, 8.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, 6.40e+04, 6.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, 8.10e+04, 7.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, 7.30e+04, 7.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, 5.70e+04, 5.00e+00]])

In [39]:
# Isntanciando o modelo de regressão linear
Modelo3 = linear_model.LinearRegression()

In [40]:
# Treinando o modelo
Modelo3.fit(X,y)

LinearRegression()

In [41]:
# Resultado do modelo
Modelo3.score(X,y)

0.9355612290620992

Análise do resultado: 
93,5% da variação da target(Preco_Venda) pode ser explicada pelas demais variáveis

# Salva o modelo treinado

In [48]:
joblib.dump(Modelo3, 'modelo3_treinado.pkl')

['modelo3_treinado.pkl']